### ORM

In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///Exc_180712.db',echo=True)
from sqlalchemy.ext.declarative import declarative_base

In [3]:
###base / autobase
Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String, ForeignKey

In [5]:
### relationship_180712
from sqlalchemy.orm import relationship

In [6]:
class Artist(Base):
    __tablename__ = 'artist'
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    album = relationship("Album", back_populates="artist")
    
    def __repr__(self):
        return "<T' artist(name= '%s')>" % (self.name)

class Album(Base):
    __tablename__ = 'album'
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey('artist.id'))
    artist = relationship("Artist", back_populates="album", uselist=False)

class Genre(Base):
    __tablename__ = 'genre'
    
    id = Column(Integer, primary_key = True)
    name = Column(String)

class Track(Base):
    __tablename__ = 'track'
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column(Integer, ForeignKey('album.id'))
    genre_id = Column(Integer, ForeignKey('genre.id'))

In [7]:
Base.metadata.create_all(engine)

2018-07-12 10:51:16,509 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-12 10:51:16,514 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:51:16,515 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-12 10:51:16,516 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:51:16,517 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-12 10:51:16,518 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:51:16,520 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-12 10:51:16,520 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:51:16,521 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-12 10:51:16,522 INFO sqlalchemy.engine.base.Engine ()
2018-07-12 10:51:16,523 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-12 10:51:16,523 INFO sqlalchemy.engine.base.Engine ()


In [8]:
artist1 = Artist(name = 'Led Zepplin')
artist2 = Artist(name = 'AC/DC')

In [9]:
artist1

<T' artist(name= 'Led Zepplin')>

In [18]:
artist1.album = [Album(title = "IV"), Album(title = "Who Made Who")]

2018-07-12 10:51:58,228 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 10:51:58,229 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id 
FROM artist 
WHERE artist.id = ?
2018-07-12 10:51:58,229 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-12 10:51:58,231 INFO sqlalchemy.engine.base.Engine UPDATE artist SET name=? WHERE artist.id = ?
2018-07-12 10:51:58,232 INFO sqlalchemy.engine.base.Engine ('xyz', 1)
2018-07-12 10:51:58,235 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-12 10:51:58,236 INFO sqlalchemy.engine.base.Engine (1,)


In [30]:
print(artist1.album[0].id)

2018-07-12 10:56:17,207 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 10:56:17,208 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-12 10:56:17,209 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-12 10:56:17,210 INFO sqlalchemy.engine.base.Engine SELECT album.id AS album_id, album.title AS album_title, album.artist_id AS album_artist_id 
FROM album 
WHERE ? = album.artist_id
2018-07-12 10:56:17,211 INFO sqlalchemy.engine.base.Engine (1,)
3


In [13]:
from sqlalchemy.orm import sessionmaker

In [14]:
Session = sessionmaker(bind = engine)
session = Session()

In [15]:
session.add_all([artist1, artist2])
session.commit()

2018-07-12 10:51:29,753 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-12 10:51:29,755 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-12 10:51:29,756 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2018-07-12 10:51:29,759 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-12 10:51:29,759 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2018-07-12 10:51:29,761 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-12 10:51:29,762 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2018-07-12 10:51:29,764 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-12 10:51:29,764 INFO sqlalchemy.engine.base.Engine ('Who Made Who', 1)
2018-07-12 10:51:29,766 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
artist1.name = 'xyz'
## dirty: 미반영된 수정사항
session.dirty

IdentitySet([<T' artist(name= 'xyz')>])

In [17]:
## new: 새로운 사항
session.new

IdentitySet([])

In [ ]:
## dirty 한 내용 반영 (수정)
session.commit()

In [ ]:
session.add_all([
    Album(title = 'IV', artist_id = artist1.id),
    Album(title = 'Who Made Who', artist_id = artist2.id),
])

session.add_all([
    Genre(name = "Rock"), Genre(name = "Metal")
])

session.commit()

In [ ]:
album1 = session.query(Album).filter(Album.artist_id == artist1.id).one()
album2 = session.query(Album).filter(Album.artist_id == artist2.id).one()

genre1 = session.query(Genre).filter(Genre.name =='Rock').filter(Genre.id ==1).one()
genre2 = session.query(Genre).filter(Genre.name =='Metal').filter(Genre.id ==2).one()